In [1]:
import os, sys

recsys_cf = os.environ.get("RECSYS_STORY")
sys.path.append(recsys_cf)
os.chdir(recsys_cf)

from lib.tools import *

In [2]:
from urllib.parse import urlparse

# Spark 

In [3]:
SPARK_CONFIG

{'kerberos_auth': True,
 'numofcores': 12,
 'numofinstances': 12,
 'process_label': 'RECSYS_STORY',
 'replication_num': 1}

In [4]:
sp = spark(**{'kerberos_auth': True,
 'numofcores': 7,
 'numofinstances': 7,
 'process_label': 'RECSYS_STORY',
 'replication_num': 1})
hive = sp.sql

###### Load visit part
Сейчас в тестовом датасете содержатся данные за одну неделю, в дальнейшем - посмотрим, на 

In [6]:
%%time
visit = load_table(SBX_TEAM_DIGITCAMP, VISIT, hive).cache()

CPU times: user 1.48 ms, sys: 3.3 ms, total: 4.78 ms
Wall time: 3.23 s


In [7]:
visit.show()

,visitNumber,cid,commonSegmentoUID,sbbolUserId,hitPagePath,hitPageTitle,sessionStartTime,sessionDate,hitType,hitNumber,hitTime,eventCategory,eventAction,eventLabel,hitPageHostName,ctl_loading,sessionId,deviceIsMobile,deviceMobileDeviceModel,deviceDeviceCategory,geoRegion,geoCity
0,1,2044466704.1626025890,None,None,/ru/person,Частным клиентам — СберБанк,1626025890,2021-07-11,EVENT,3,487,SITE_Widget_TabSegmentContainerFlex,segment_set,-2,www.sberbank.ru,10049073,1626025890,False,None,desktop,(not set),(not set)
1,4,655506626.1623512191,W5c8iv1cUkMK,None,/ipoteka/programs/existing-apartments?from=landing,"Ипотека на готовое жилье в Ростове-на-Дону от 0,1% от СберБанка онлайн в 2021 году, взять ипотечный кредит на готовую квартиру в банке: условия, ставки, срок до 30 лет - ДомКлик",1626011900,2021-07-11,PAGE,1,0,None,None,None,rostov-na-donu.domclick.ru,10049073,1626011900,False,None,desktop,Rostov Oblast,Taganrog
2,1,1708587183.1626019981,None,None,/ru/person/bank_cards/debit/sberkarta_msc,СберКарта — СберБанк,1626019981,2021-07-11,EVENT,7,23091,SITE_Widget_DebitCardOrder,reachSessionStorage,.....,www.sberbank.ru,10049073,1626019981,False,None,desktop,Republic of Tatarstan,Kazan
3,1,895598413.1625992080,None,None,/ru/person/bank_cards/debit/sberkarta_social,СберКарта Мир для пособий и пенсии — СберБанк,1625992079,2021-07-11,EVENT,87,1551857,SITE_Person_bank_cards.debit.sberkarta_social,click_any,"Театральная улица, 23_ymaps-2-1-78-islets_serp-item__title",www.sberbank.ru,10049073,1625992079,False,None,desktop,Donetsk Oblast,Horlivka
4,1,543458166.1625952886,None,None,/mortgage/calc?isolated=true,Домклик - Ипотека от СберБанка Онлайн,1625952888,2021-07-11,EVENT,13,54288,DOMCLICK_.mortgage.calc,calculated,MortgageLoan,ipoteka.domclick.ru,10049073,1625952888,False,None,desktop,Moscow Oblast,Dubna
5,1,1976186634.1625972356,None,None,/mortgage/calc?prod=3&cost=5700000&dep=1425000&term=20&dk=true&ins=true&married=false&onlineReg=true&developerDiscount=false&isolated=true&hideSaveButton=true,Домклик - Ипотека от СберБанка Онлайн,1625972358,2021-07-11,EVENT,13,79519,DOMCLICK_.mortgage.calc,calculated,MortgageLoan,ipoteka.domclick.ru,10049073,1625972358,False,None,desktop,Primorsky Krai,Vladivostok
6,1,889258704.1625980408,None,None,/mortgage/application-mobile.html?appsflyer_id=1599896473972-1558876621896212695,Домклик - Ипотека от СберБанка Онлайн,1625980408,2021-07-11,EVENT,24,732698,DOMCLICK_.mortgage.calc.,calculated,MortgageLoan,ipoteka.domclick.ru,10049073,1625980408,False,None,desktop,Chelyabinsk Oblast,Chelyabinsk
7,1,1221548771.1626030019,None,None,/mortgage/application-mobile.html?appsflyer_id=1621171629903-3116869100971508247,Домклик - Ипотека от СберБанка Онлайн,1626030019,2021-07-11,EVENT,22,94309,DOMCLICK_.mortgage.calc.,calculated,MortgageLoan,ipoteka.domclick.ru,10049073,1626030019,False,None,desktop,Moscow,Moscow
8,442,1591118459.1558497639,None,3c25f095-d1fe-fc84-e054-90e2ba85d0c8,/main,СберБизнес,1625960650,2021-07-11,EVENT,27,82715,[operations]: offers,impression-story,[placement: main[main_feed]][container: 8bb362eb-9c78-4307-b4ed-5df94bab41fc][bc: interaction][product: none],sbi.sberbank.ru,10049073,1625960650,False,None,desktop,Sverdlovsk Oblast,Yekaterinburg
9,3,985565301.1625490595,None,None,/profile,СберБанк Онлайн,1626002150,2021-07-11,EVENT,26,169220,SBOL_WEB_profile,Страница профиля,Открытие,web5.online.sberbank.ru,10049073,1626002150,False,None,desktop,Valencian Community,(not set)


In [ ]:
visit.show()

In [4]:
visit = hive.sql("""select *
                 from cap_external_google_dda_external_google_analytics.visit
                 where 1 = 1
                   and sessiondate >= '2021-06-28'
                   and sessiondate <= '2021-07-04'
                   and hitPageHostName = 'sbi.sberbank.ru'""")

In [5]:
visit_cut = visit.filter("hitPageHostName = 'sbi.sberbank.ru'")\
                 .select("sbbolUserId",
                         "sbbolorgguid",
                         "hitPagePath",
                         "hitPageTitle",
                        "sessionId",
                        "sessionStartTime",
                        "sessionDate",
                        "hitType",
                        "hitTime",
                        "eventCategory",
                        "eventAction",
                        "eventLabel",
                        "hitPageHostName",
                        "sessionId")

In [6]:
visit_cut.cache()

DataFrame[sbbolUserId: string, sbbolorgguid: string, hitPagePath: string, hitPageTitle: string, sessionId: bigint, sessionStartTime: bigint, sessionDate: string, hitType: string, hitTime: bigint, eventCategory: string, eventAction: string, eventLabel: string, hitPageHostName: string, sessionId: bigint]

In [7]:
visit_cut.count()

605274831

###### Отчистка путей от мусора 

In [8]:
@udf(stypes.StringType())
def get_url_path(url):
        try:
            url_parse = urlparse(url)
        except Exception as e:
            print(e)
            return None

        return url_parse.path

In [9]:
visit_cols = visit_cut.withColumn("hitPagePath", get_url_path(f.col("hitPagePath")))\
                      .filter(f.col("hitPagePath") != "/" )

###### приведение времени хита к общему времени 

In [10]:
visit_ts = visit_cols.withColumn("timestamp", 
                                 (f.col("sessionStartTime") + f.col("hitTime") / 1000).cast(stypes.LongType())) \
                     .drop("sessionStartTime", "hitTime")

In [11]:
visit_ts.show(3)

,sbbolUserId,sbbolorgguid,hitPagePath,hitPageTitle,sessionId,sessionDate,hitType,eventCategory,eventAction,eventLabel,hitPageHostName,sessionId,timestamp
0,19046ac3-39dd-4565-b69d-52e922690187,f0dd1036-cabe-4cb5-9432-2d904200b53e,/accounts-payments/account/:accountId,СберБизнес,1622969141,2021-06-06,EVENT,[std]: accounts-payments,show,[accountsandpayments paymentstable]: [showed],sbi.sberbank.ru,1622969141,1622969171
1,3c25f078-4d98-fc84-e054-90e2ba85d0c8,77eb1019-e8f0-48cd-a9a5-ef6e49797539,/main,СберБизнес,1622949136,2021-06-06,EVENT,[std]: layout,menu fetching,"[layout.primarymenu]: [count: 16, visible: [menu_create_section, menu_import, menu_help, menu_accounts_and_payments, menu_statements_and_reports, menu_counterparties, menu_templates_and_autopayments, menu_inquiries, menu_item_korpkarta, menu_item_zarplata, menu_item_kredit, menu_item_merchant-acquiring, menu_item_deposit, menu_item_e_inv, menu_item_currency_control, menu_item_derivatives], hidden: []]",sbi.sberbank.ru,1622949136,1622949175
2,3c25f07d-1d23-fc84-e054-90e2ba85d0c8,2bb9dc6a-6ecb-4951-b2ba-9332c173f973,/regular-and-templates/templates,СберБизнес - версия: 41.000.04_0005,1623008267,2021-06-06,EVENT,[operations]: offers,impression-story,[placement: ruble_payment[context]][container: a46ddad1-4e25-72cb-e054-90e2ba8333f8][bc: adv_bank][product: refundvat],sbi.sberbank.ru,1623008267,1623008637


In [12]:
visit_ts.cache()

DataFrame[sbbolUserId: string, sbbolorgguid: string, hitPagePath: string, hitPageTitle: string, sessionId: bigint, sessionDate: string, hitType: string, eventCategory: string, eventAction: string, eventLabel: string, hitPageHostName: string, sessionId: bigint, timestamp: bigint]

In [13]:
visit_ts.count()

605274169

###### Дропаем юзеров без сббол ид 

###### подтягиваем инн юзеров 

In [14]:
customer_sbbol = load_table(SBX_TEAM_DIGITCAMP, "ga_ma_customer_sbbol", hive)

In [18]:
customer_sbbol.count()

5268797

In [15]:
customer_sbbol.show(3)

,CU_ID_SBBOL,SBBOL_NODE,CU_INN,CU_KPP,CU_OGRN,CU_OKPO,CU_ADDR_COMP,CU_NAME_COMP,CU_NAL_STATUS_FLAG,CU_REG_DATE,SUSPENDED,FINANCEBLOCK,PARENT_EKS_ID,CHILD_EKS_ID,LOCKED,ANNULED,REPLICATIONGUID
0,3864241,1,235201488511,None,319237500136816,0152765395,"353539, Краснодарский край, п.СТРЕЛКА, Темрюкский р-он, ул. Космодемьянской, дом 20",Индивидуальный предприниматель БЫДРУ ИРИНА ВИКТОРОВНА,1.0,2019-04-18,0,1,None,None,0,1,6be57744-e1dd-4bd4-a817-761ba9602f08
1,5176479,1,235201489890,None,321237500098442,2006687123,"353555, Краснодарский край, ст-ца Тамань, ул. Пушкина, дом 35",Индивидуальный предприниматель ПАВЛЕНКО ВАСИЛИЙ СЕРГЕЕВИЧ,1.0,2021-03-24,0,0,8202343246741,8202343246741,0,0,aa60b531-4066-4fad-996b-8601fedc74c4
2,3245353,1,235201493977,None,311235212500063,None,None,ИП Фефелов Владимир Александрович,1.0,2011-05-05,0,1,None,None,0,1,cf3c306e-12f0-449e-a7e2-efc1f46bc58f


In [16]:
visit_ts = customer_sbbol.select("REPLICATIONGUID", f.col("CU_INN").alias("inn"))\
                         .join(visit_ts,
                               customer_sbbol.REPLICATIONGUID == visit_ts.sbbolorgguid,
                               "inner")\
                         .drop("REPLICATIONGUID", "sbbolUserId", "sbbolorgguid")\
                         .distinct()

In [17]:
visit_ts.show(3)

,inn,hitPagePath,hitPageTitle,sessionId,sessionDate,hitType,eventCategory,eventAction,eventLabel,hitPageHostName,sessionId,timestamp
0,5009097772,/main,СберБизнес,1622628111,2021-06-02,EVENT,[operations]: offers,impression-story,[placement: main|interaction[main_feed]][container: 365247d0-bd74-4e25-a5d9-92ac1765dc99][bc: none][product: none][platform: ufs],sbi.sberbank.ru,1622628111,1622628334
1,1655437749,/accounts-payments/account/:accountId,СберБизнес - версия: 41.000.01_0003,1622462000,2021-05-31,EVENT,[std]: payments,show,[ruble_payment_edit]: [ruble_payment_edit__rub_comm],sbi.sberbank.ru,1622462000,1622463120
2,1655437749,/login/check-otp,СберБизнес,1623153371,2021-06-08,EVENT,[std]: login,show,[checkotpform]: [sms-auto-check-mode],sbi.sberbank.ru,1623153371,1623153372


In [18]:
visit_ts.select("inn").distinct().count()

1046460

###### дропаем юзеров без инн 

In [19]:
visit_ts = visit_ts.filter(~f.isnull("inn"))

###### Оставляем в кликстриме только нужные события
В данном случае решил оставить все клики и нажатия на сторисы, по которым мы будем скориться

In [20]:
STORY_CLICK = "hitType = 'EVENT' AND eventCategory = '[operations]: offers' AND eventAction  = 'open card'"

USER_CLICK = "eventAction = 'click'"

In [21]:
clickstream_filter = visit_ts.filter(STORY_CLICK + " OR " + USER_CLICK)

In [22]:
clickstream_filter.cache()

DataFrame[inn: string, hitPagePath: string, hitPageTitle: string, sessionId: bigint, sessionDate: string, hitType: string, eventCategory: string, eventAction: string, eventLabel: string, hitPageHostName: string, sessionId: bigint, timestamp: bigint]

In [23]:
%%time
clickstream_filter.show(3)

CPU times: user 2.73 s, sys: 4.85 s, total: 7.58 s
Wall time: 1min 52s


,inn,hitPagePath,hitPageTitle,sessionId,sessionDate,hitType,eventCategory,eventAction,eventLabel,hitPageHostName,sessionId,timestamp
0,3250533832,/inquiries,СберБизнес - версия: 41.000.03_0001,1622788830,2021-06-04,EVENT,[std]: main,click,"[layout.headergeneralsbbol3]: [open burger, novelty: false]",sbi.sberbank.ru,1622788830,1622791845
1,3250533832,/statements/daily/CALCULATED_RUB/:statementId,СберБизнес - версия: 41.000.01_0003,1622445075,2021-05-31,EVENT,[std]: accounts statements,click,[account daily statement page]: [accounttransactions:action.export_account_transactions_as_file],sbi.sberbank.ru,1622445075,1622445131
2,3250533832,/accounts-payments/account/:accountId,СберБизнес - версия: 41.000.04_0005,1623063692,2021-06-07,EVENT,[std]: main,click,"[layout.headergeneralsbbol3]: [open burger, novelty: false]",sbi.sberbank.ru,1623063692,1623067943


In [24]:
%%time
clickstream_filter.distinct().count()

CPU times: user 48.6 ms, sys: 43.8 ms, total: 92.4 ms
Wall time: 1min 30s


107172789

In [25]:
clickstream_uniq = clickstream_filter.distinct().cache()

###### Выделение продукта из экшена 

Взял из кликов также события с продуктовыми метками, тк они в основном падают со странице с магазином продуктов, по суте - те же сторисы, только в кликстриме по-другому называются

такими метками отмечаются продукты, на которые юзеры попали через вкладку все продукты возможно, что метка activated - если продукт подключен, not - activated - продукт не подключен

In [26]:
pattern1 = 'product\: [^]]*]{1}'
pattern2 = 'product\: [^,]*,{1}'
clickstream_product = clickstream_uniq.withColumn('product',f.regexp_extract("eventLabel", pattern1, 0))\
                                      .withColumn('product',
                                                  f.when(f.col("product").like("%,%"),
                                                         f.regexp_extract("eventLabel", pattern2, 0))\
                                                   .otherwise(f.col("product")))

In [27]:
clickstream_product = clickstream_product.withColumn("product",
                                                            f.col("product").substr(f.lit(10), f.length("product") - 10))

In [28]:
clickstream_product.cache()

DataFrame[inn: string, hitPagePath: string, hitPageTitle: string, sessionId: bigint, sessionDate: string, hitType: string, eventCategory: string, eventAction: string, eventLabel: string, hitPageHostName: string, sessionId: bigint, timestamp: bigint, product: string]

In [29]:
clickstream_product.show(3)

,inn,hitPagePath,hitPageTitle,sessionId,sessionDate,hitType,eventCategory,eventAction,eventLabel,hitPageHostName,sessionId,timestamp,product
0,3250533832,/inquiries,СберБизнес - версия: 41.000.03_0001,1622788830,2021-06-04,EVENT,[std]: main,click,"[layout.headergeneralsbbol3]: [open burger, novelty: false]",sbi.sberbank.ru,1622788830,1622791845,
1,3250533832,/statements/daily/CALCULATED_RUB/:statementId,СберБизнес - версия: 41.000.01_0003,1622445075,2021-05-31,EVENT,[std]: accounts statements,click,[account daily statement page]: [accounttransactions:action.export_account_transactions_as_file],sbi.sberbank.ru,1622445075,1622445131,
2,3250533832,/accounts-payments/account/:accountId,СберБизнес - версия: 41.000.04_0005,1623063692,2021-06-07,EVENT,[std]: main,click,"[layout.headergeneralsbbol3]: [open burger, novelty: false]",sbi.sberbank.ru,1623063692,1623067943,


In [30]:
clickstream_product.select("product").distinct().count()

111

In [31]:
clickstream_product.select("inn",
                           "product")\
                   .distinct()\
                   .groupBy("product")\
                   .agg(f.count("inn"))\
                   .orderBy(f.col("count(inn)").desc())\
                   .show(15)

,product,count(inn)
0,,1038530
1,none,29393
2,e-acquiring,5254
3,kredit,5125
4,merchant-acquiring,4377
5,smart_credit,4178
6,refundvat,3621
7,zarplata,3547
8,rabota_ru,2656
9,sppu,2604


In [32]:
clickstream_final = clickstream_product.select('inn',
                                               'timestamp',
                                               'hitPagePath',
                                               "hitPageTitle",
                                               'eventCategory',
                                               'eventAction',
                                               'eventLabel',
                                               'product').distinct()

In [33]:
clickstream_final.show(3)

,inn,timestamp,hitPagePath,hitPageTitle,eventCategory,eventAction,eventLabel,product
0,470707811305,1623239609,/accounts-payments,СберБизнес,[std]: payments,click,[ruble_payment_edit]: [action.save],
1,6220008542,1623318711,/main,СберБизнес,[std]: main,click,"[layout.headergeneralsbbol3]: [open burger, novelty: true]",
2,5260170451,1623412583,/accounts-payments/account/:accountId,СберБизнес,[std]: payments,click,[ruble_payment_edit]: [action.save],


In [34]:
clickstream_final.count()

107172232

###### Сохранение кликстрима 

In [35]:
RECSYS_STORY_CLICKSTREAM

'RECSYS_STORY_CLICKSTREAM'

In [36]:
drop_table(SBX_TEAM_DIGITCAMP, RECSYS_STORY_CLICKSTREAM, hive)  

In [37]:
%%time
create_table_from_df(SBX_TEAM_DIGITCAMP, RECSYS_STORY_CLICKSTREAM, clickstream_final, hive)

CPU times: user 86.1 ms, sys: 62.8 ms, total: 149 ms
Wall time: 1min 27s


###### Статистика по кликстриму 

Все эвенты из кликстрима

In [46]:
load_table(SBX_TEAM_DIGITCAMP, RECSYS_STORY_CLICKSTREAM, hive).show(3)

,inn,timestamp,hitPagePath,hitPageTitle,eventCategory,eventAction,eventLabel,product
0,470707811305,1623239609,/accounts-payments,СберБизнес,[std]: payments,click,[ruble_payment_edit]: [action.save],
1,6220008542,1623318711,/main,СберБизнес,[std]: main,click,"[layout.headergeneralsbbol3]: [open burger, novelty: true]",
2,5260170451,1623412583,/accounts-payments/account/:accountId,СберБизнес,[std]: payments,click,[ruble_payment_edit]: [action.save],


In [38]:
%%time
clickstream_product.count()

CPU times: user 16.1 ms, sys: 24.7 ms, total: 40.7 ms
Wall time: 358 ms


107172789

In [39]:
%%time
clickstream_product.groupby("sessionDate").agg(f.count("inn")).show()

CPU times: user 44 ms, sys: 46.6 ms, total: 90.6 ms
Wall time: 2.02 s


,sessionDate,count(inn)
0,2021-06-11,10294367
1,2021-06-08,9503928
2,2021-06-05,1041787
3,2021-06-06,1052803
4,2021-06-01,10765396
5,2021-06-10,9657617
6,2021-06-14,2509707
7,2021-06-09,9253116
8,2021-06-02,9269363
9,2021-06-03,9359251


In [40]:
clickstream_product.select("inn").distinct().count()

1038609

статистика по кликам по сторям

In [41]:
clickstream_story = clickstream_product.filter("eventCategory = '[operations]: offers' and product <> 'none'")

In [42]:
%%time
clickstream_story.select("inn").distinct().count()

CPU times: user 40.9 ms, sys: 56.4 ms, total: 97.2 ms
Wall time: 5.61 s


26495

In [43]:
clickstream_product_stat = clickstream_product.filter("product <> '' and product <> 'none'")

In [44]:
clickstream_product_stat.select("inn").distinct().count()

46489